# Setting

In [1]:
import pandas as pd
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

# 前処理

In [ ]:
#Enumerate Homeplanet
train = df_train.copy()

title_mapping = {"Europa": 1, "Earth": 2, "Mars": 3}
train['HomePlanet'] = train['HomePlanet'].map(title_mapping)
train['HomePlanet'] = train['HomePlanet'].fillna(0)

train[['Cabin_1', 'Cabin_2', 'Cabin_3']] = train['Cabin'].str.split('/', expand=True)
train.drop('Cabin', axis=1, inplace=True)

cabin_1_mapping = {"B": 1, "F": 2, "A": 3, "G": 4, "E": 5, "D": 6, "C": 7, "T": 8}
train['Cabin_1'] = train['Cabin_1'].map(cabin_1_mapping)
train['Cabin_1'] = train['Cabin_1'].fillna(0)

cabin_3_mapping = {"P": 1, "S": 2}
train['Cabin_3'] = train['Cabin_3'].map(cabin_3_mapping)
train['Cabin_3'] = train['Cabin_3'].fillna(0)

train['Cabin_2'] = train['Cabin_2'].fillna(9999)
train['Cabin_2'] = train['Cabin_2'].astype(int) + 1
train['Cabin_2'] = train['Cabin_2'].replace(10000, 0)


train['Fare'] = train[['FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
cols = ['FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck']
for col in cols:
    train[col] = train[col].fillna(train[col].median())


train.loc[train["Transported"] == 1, "Age"] = train[train["Transported"] == 1]["Age"].fillna(train[train["Transported"] == 1]["Age"].mean())
train.loc[train["Transported"] == 0, "Age"] = train[train["Transported"] == 0]["Age"].fillna(train[train["Transported"] == 0]["Age"].mean())

print(train["Age"].mean())

train.drop('Destination', axis=1, inplace=True)

vip_mapping = {False: 1, True: 2}
train['VIP'] = train['VIP'].map(vip_mapping)
train['VIP'] = train['VIP'].fillna(0)

cryo_mapping = {False: 1, True: 2}
train['CryoSleep'] = train['CryoSleep'].map(cryo_mapping)
train['CryoSleep'] = train['CryoSleep'].fillna(0)

# 学習用の目的変数を保存
y = train['Transported']

train.drop(['Name', 'Transported', 'PassengerId'], axis = 1, inplace = True)

In [5]:
#Cleaning Tes
test = df_test.copy()
title_mapping = {"Europa": 1, "Earth": 2, "Mars": 3}
test['HomePlanet'] = test['HomePlanet'].map(title_mapping)
test['HomePlanet'] = test['HomePlanet'].fillna(0)

test[['Cabin_1', 'Cabin_2', 'Cabin_3']] = test['Cabin'].str.split('/', expand=True)
test.drop('Cabin', axis=1, inplace=True)

cabin_1_mapping = {"B": 1, "F": 2, "A": 3, "G": 4, "E": 5, "D": 6, "C": 7, "T": 8}
test['Cabin_1'] = test['Cabin_1'].map(cabin_1_mapping)
test['Cabin_1'] = test['Cabin_1'].fillna(0)

cabin_3_mapping = {"P": 1, "S": 2}
test['Cabin_3'] = test['Cabin_3'].map(cabin_3_mapping)
test['Cabin_3'] = test['Cabin_3'].fillna(0)

test['Cabin_2'] = test['Cabin_2'].fillna(9999)
test['Cabin_2'] = test['Cabin_2'].astype(int) + 1
test['Cabin_2'] = test['Cabin_2'].replace(10000, 0)


test['Fare'] = test[['FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
cols = ['FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck']
for col in cols:
    test[col] = test[col].fillna(test[col].median())

test['Age'] = test['Age'].fillna(28.827968418500845)

test.drop('Destination', axis=1, inplace=True)

vip_mapping = {False: 1, True: 2}
test['VIP'] = test['VIP'].map(vip_mapping)
test['VIP'] = test['VIP'].fillna(0)

cryo_mapping = {False: 1, True: 2}
test['CryoSleep'] = test['CryoSleep'].map(cryo_mapping)
test['CryoSleep'] = test['CryoSleep'].fillna(0)




test.drop(['Name', 'PassengerId'], axis = 1, inplace = True)

In [6]:
data = df_train.copy()

# Step 1: Check for missing values
missing_values = data.isnull().sum()

# Step 2: Data type conversion and initial transformations
# Convert CryoSleep and VIP to boolean
data['CryoSleep'] = data['CryoSleep'].map({'True': True, 'False': False})
data['VIP'] = data['VIP'].map({'True': True, 'False': False})

# Step 3: Feature engineering
# Split Cabin into deck, num, and side
data[['Deck', 'Num', 'Side']] = data['Cabin'].str.split('/', expand=True)

# Split PassengerId into group and number
data[['Group', 'GroupNum']] = data['PassengerId'].str.split('_', expand=True)

# Create a new feature: Total expenditure
data['TotalExpenditure'] = data[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)

# Step 4: Check target distribution
target_distribution = data['Transported'].value_counts(normalize=True)

# Display the processed dataset overview and results
processed_overview = {
    "Missing Values": missing_values,
    "Target Distribution": target_distribution,
    "Columns After Feature Engineering": data.columns.tolist()
}

In [7]:
# trainとtestを保存する
train.to_csv('train_v2.csv', index=False)
test.to_csv('test_v2.csv', index=False)

In [ ]:
train.head()

## モデル構築〜予測

In [ ]:
!pip install catboost
!pip install lightgbm
!pip install xgboost
!pip install scikit-learn

In [ ]:
import catboost
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
cat = catboost.CatBoostClassifier()
lgbm = lgb.LGBMClassifier()
xgb = xgb.XGBClassifier()
rfc = RandomForestClassifier()
lr = LogisticRegression()
knn = KNeighborsClassifier()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.3, random_state=42)
y_train = y_train.astype(int)

# 形状確認
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# 学習
cat.fit(X_train, y_train)
lgbm.fit(X_train, y_train)
xgb.fit(X_train, y_train)
rfc.fit(X_train, y_train)
lr.fit(X_train, y_train)
knn.fit(X_train, y_train)

In [14]:
# 予測
cat_pred = cat.predict(X_test)
lgbm_pred = lgbm.predict(X_test)
xgb_pred = xgb.predict(X_test)
rfc_pred = rfc.predict(X_test)
lr_pred = lr.predict(X_test)
knn_pred = knn.predict(X_test)

In [ ]:
# それぞれのモデルのaccuracy_scokreをリストにまとめる
models = [rfc, lr, knn, cat, lgbm, xgb]
scores = []
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    scores.append(score)

# scoresを降順にソート
sorted_scores = sorted(scores, reverse=True)

In [ ]:
# scoresをモデル名とともに表示

# モデル名とスコアの対応付けを辞書で作成
model_names = ['RandomForestClassifier', 'LogisticRegression', 'KNN', 'CatBoost', 'lightGBM', 'XGBoost',]
model_scores = dict(zip(model_names, scores))

# スコアを降順にソートし、モデル名とともに表示
for model_name, score in sorted(model_scores.items(), key=lambda item: item[1], reverse=True):
    print(f'{model_name}: {score}')

In [29]:
# テストデータで予測
test_pred = lgbm.predict(test)

In [30]:
# test_predで0の場合はFalse、1の場合はTrueに変換
test_pred = test_pred.astype(bool)

In [31]:
test_pred.shape

(4277,)

## 提出データ作成

In [33]:
sample = pd.read_csv('sample_submission.csv')
# sampleデータのTransported列にtest_predを代入
sample['Transported'] = test_pred

In [35]:
# データ出力
sample.to_csv('submission.csv', index=False)